In [1]:
import sys
import copy 
import random
import pandas as pd
import numpy as np
import torch
import tqdm.auto as tqdm
from pathlib import Path

from typing import *
from enum import Enum
from dataclasses import dataclass

/Users/tolya/.miniconda3/envs/XCS234/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("research")

In [4]:
import os,sys
sys.path.insert(0, str(Path.cwd().parent))

In [5]:
import utils.logging
utils.logging.setup(debug=False)

### Narde rules
https://www.bkgm.com/variants/Narde.html

In [6]:
class Game:
    class Step(Enum):
        IDLE = "idle"
        ROLL = "roll"
        TURN = "trun"
        FINISHED = "finished"

    def __init__(self, seed: Optional[int] = None):
        self.seed = random.randint(0, 2**32-1) if seed is None else seed
        logger.debug(f"new game started, seed={self.seed}")
        random.seed(self.seed)
        np.random.seed(self.seed)
        self.board = np.zeros(24, dtype=int)
        self.board[0] = 15
        self.board[12] = -15
        self.dice = [0, 0]
        self.home = [0, 0]
        self.last = [0, 12]
        self.pturn = 0
        self.t = 0
        self.head_moves = 0
        self.step : Game.Step = Game.Step.IDLE

    def _get_next_idx(self, src_idx: int, steps: int) -> int:
        dst_idx = src_idx + steps
        if dst_idx > 24:
            dst_idx = dst_idx % 25 + 1
        if dst_idx < 1:
            dst_idx = 24 + dst_idx
        return dst_idx
    
    def _get_head(self, player: Optional[int] = None) -> int:
        player = player if player is not None else self.pturn
        return 1 if player == 0 else 13
    
    def _get_user_sign(self, player: Optional[int] = None) -> int:
        player = player if player is not None else self.pturn
        return 1 if player == 0 else -1

    def _can_move_home(self):
        counter = self.home[self.pturn]
        home = range(19, 25) if self.pturn == 0 else range(7, 13)
        for idx in home:
            if self._has_checkers(idx, player=self._cur_player):
                counter += self._get_checkers(idx, player=self._cur_player)
        return counter == 15

    def _is_move_home(self, src_idx: int, steps: int) -> bool:
        dst_idx = self._get_next_idx(src_idx, steps)
        return (self.pturn == 0 and dst_idx < src_idx) or \
               (self.pturn == 1 and src_idx <= 12 and dst_idx > 12)
    
    @property
    def _opponent(self) -> int:
        return 1 if self.pturn == 0 else 0
    
    @property
    def _cur_player(self) -> int:
        return self.pturn
    
    def _has_checkers(self, pos: int, player: Optional[int] = None):
        return self._get_checkers(pos, player) > 0
    
    def _get_checkers(self, pos: int, player: Optional[int] = None):
        player = player if player is not None else self.pturn
        return self._get_user_sign(player) * self.board[pos-1]
    
    def _find_prime(self, pos: int) -> Optional[Tuple[int, int]]:
        seq = 1
        result = [pos, pos]
        next_ptr = self._get_next_idx(pos, 1)
        while seq < 6 and self._has_checkers(next_ptr, player=self._cur_player):
            seq += 1
            result[1] = next_ptr
            next_ptr = self._get_next_idx(next_ptr, 1)
        prev_ptr = self._get_next_idx(pos, -1)
        while seq < 6 and self._has_checkers(prev_ptr, player=self._cur_player):
            seq += 1
            result[0] = prev_ptr
            prev_ptr = self._get_next_idx(prev_ptr, -1)
        return tuple(result) if seq == 6 else None
    
    def _is_blocking_prime(self, dst_idx: int) -> bool:
        prime_range = self._find_prime(dst_idx)
        if prime_range:
            prime_end_idx = prime_range[1]
            for step in range(1, 24):
                search_idx = self._get_next_idx(prime_end_idx, step)
                if self._get_head(self._opponent) == search_idx:
                    return True # we reached other player's home without finding it's checkers
                if self._has_checkers(pos=search_idx, player=self._opponent):
                    break
        return False
    
    def _check_move(self, src_idx: int, steps: int):
        dst_idx = self._get_next_idx(src_idx, steps)
        if self.step != Game.Step.TURN:
            raise RuntimeError("invalid action")
        if not (1 <= src_idx <= 24):
            raise RuntimeError("invalid position")
        if not self._has_checkers(src_idx, player=self._cur_player):
            raise RuntimeError(f"no checkers at position {src_idx}")
        if self._has_checkers(dst_idx, player=self._opponent):
            raise RuntimeError(f"can't move to position {dst_idx}")
        if steps not in self.dice:
            raise RuntimeError(f"no dice with value {steps}")
        if self._is_move_home(src_idx, steps):
            if not self._can_move_home():
                raise RuntimeError(f"not all checkers are at finishing table")
        if self._get_head() == src_idx and self.head_moves > 0:
            if self.head_moves > 1 or self.dice[0] != self.dice[1]:
                raise RuntimeError(f"can't make any more head moves")
        if self._is_blocking_prime(dst_idx):
            raise RuntimeError(f"can't form a blocking prime")
        # TODO: If player can play one number but not both, they must play the higher one

    def _render_player(self, player: Optional[int] = None, lower: bool = True) -> str:
        player = player if player is not None else self.pturn
        result = "O" if player == 0 else "X"
        return result.lower() if lower else result

    def _enum_valid_moves(self) -> Iterator[Tuple[int, int]]:
        eligible_moves = []
        for src_idx in range(1, 25):
            if self._has_checkers(src_idx, player=self._cur_player):
                for steps in range(1, 7):
                    if self._is_valid_move(src_idx, steps):
                        yield (src_idx, steps)
        return eligible_moves
        
    def _is_valid_move(self, src_idx: int, steps: int) -> bool:
        try:
            self._check_move(src_idx, steps)
            return True
        except RuntimeError as e:
            return False
        
    def _can_move(self):
        try:
            next(iter(self._enum_valid_moves()))
            return True
        except StopIteration:
            return False

    def start(self, d1: int = 0, d2: int = 0) -> "Game":
        if self.step != Game.Step.IDLE:
            raise RuntimeError("invalid action")

        self.dice = [d1 or random.randint(1, 6), d2 or random.randint(1, 6)]
        while self.dice[0] == self.dice[1]:
            self.dice = [random.randint(1, 6), random.randint(1, 6)]

        self.step = Game.Step.ROLL
        if self.dice[0] > self.dice[1]:
            self.pturn = 0
        else: # self.dice[0] < self.dice[1]:
            self.pturn = 1
        return self

    def roll(self, d1: int = 0, d2: int = 0) -> "Game":
        if self.step != Game.Step.ROLL:
            raise RuntimeError("invalid action")
        self.dice = [d1 or random.randint(1, 6), d2 or random.randint(1, 6)]
        logger.debug(f"t={self.t}, p={self._render_player()} rolls {self.dice}")
        if self.dice[0] == self.dice[1]:
            self.dice += self.dice
        self.step = Game.Step.TURN
        return self

    def turn(self, src_idx: int, steps: int) -> "Game":
        dst_idx = self._get_next_idx(src_idx, steps)
        self._check_move(src_idx, steps)
        if self._is_move_home(src_idx, steps):
            logger.debug(f"t={self.t}, p={self._render_player()} moves: {src_idx}->HOME")
            self.board[src_idx-1] -= self._get_user_sign()
            self.home[self.pturn] += 1
        else:
            logger.debug(f"t={self.t}, p={self._render_player()} moves: {src_idx}-({steps})->{dst_idx}")
            self.board[src_idx-1] -= self._get_user_sign()
            self.board[dst_idx-1] += self._get_user_sign()

        if self._get_head() == src_idx:
            self.head_moves += 1

        if len(self.dice) > 2:
            self.dice.pop(self.dice.index(steps, -1))
        else:
            self.dice[self.dice.index(steps)] = 0

        if self.home[self.pturn] == 15:
            logger.debug(f"t={self.t}, game finished, p={self._render_player()} wins")
            self.step = Game.Step.FINISHED
        elif self.dice[0] == 0 and self.dice[1] == 0:
            self.step = Game.Step.ROLL
            self.pturn = self._opponent
            self.t += 1
            self.head_moves = 0
        
        return self
    
    def is_finished(self):
        return self.step == Game.Step.FINISHED
    
    def skip(self) -> "Game":
        if self.step != Game.Step.TURN:
            raise RuntimeError("invalid action")
        
        if self._can_move():
            raise RuntimeError("skip only possible when there's no moves")
        else:
            logger.debug(f"t={self.t}, p={self._render_player()} has no eligible moves, skipping")
            self.dice = [0, 0]
            self.step = Game.Step.ROLL
            self.pturn = (self.pturn + 1) % 2
            self.t += 1
        
        return self

    def __repr__(self):
        template = """
        |{oha}| 24 | 23 | 22 | 21 | 20 | 19 |{xst}| 18 | 17 | 16 | 15 | 14 | 13 |{xho}|
        |{ohb}|-----------------------------|     |-----------------------------|{xhn}|
        |{ohc}|{x1}|{w1}|{v1}|{u1}|{t1}|{s1}|     |{r1}|{q1}|{p1}|{o1}|{n1}|{m1}|{xhm}|
        |{ohd}|{x2}|{w2}|{v2}|{u2}|{t2}|{s2}|     |{r2}|{q2}|{p2}|{o2}|{n2}|{m2}|{xhl}|
        |{ohe}|{x3}|{w3}|{v3}|{u3}|{t3}|{s3}|     |{r3}|{q3}|{p3}|{o3}|{n3}|{m3}|{xhk}|
        |{ohf}|{x4}|{w4}|{v4}|{u4}|{t4}|{s4}|     |{r4}|{q4}|{p4}|{o4}|{n4}|{m4}|{xhj}|
        |{ohg}|{x5}|{w5}|{v5}|{u5}|{t5}|{s5}|     |{r5}|{q5}|{p5}|{o5}|{n5}|{m5}|{xhi}|
        |{ohh}|-----------------------------|{dcs}|-----------------------------|{xhh}|
        |{ohi}|{a5}|{b5}|{c5}|{d5}|{e5}|{f5}|     |{g5}|{h5}|{i5}|{j5}|{k5}|{l5}|{xhg}|
        |{ohj}|{a4}|{b4}|{c4}|{d4}|{e4}|{f4}|     |{g4}|{h4}|{i4}|{j4}|{k4}|{l4}|{xhf}|
        |{ohk}|{a3}|{b3}|{c3}|{d3}|{e3}|{f3}|     |{g3}|{h3}|{i3}|{j3}|{k3}|{l3}|{xhe}|
        |{ohl}|{a2}|{b2}|{c2}|{d2}|{e2}|{f2}|     |{g2}|{h2}|{i2}|{j2}|{k2}|{l2}|{xhd}|
        |{ohm}|{a1}|{b1}|{c1}|{d1}|{e1}|{f1}|     |{g1}|{h1}|{i1}|{j1}|{k1}|{l1}|{xhc}|
        |{ohn}|-----------------------------|     |-----------------------------|{xhb}|
        |{oho}| 01 | 02 | 03 | 04 | 05 | 06 |{ost}| 07 | 08 | 09 | 10 | 11 | 12 |{xha}|
        """

        pixels = {}
        for pos in range(1,25):
            x = chr(ord("a") + pos - 1)
            checkers = abs(int(self.board[pos-1]))
            pixel = self._render_player(player = 0 if self.board[pos-1] > 0 else 1)
            for y in range(1, 6):
                if checkers > 0:
                    if y < 5 or checkers == 1:
                        pixels[f"{x}{y}"] = (f"  {pixel} ")
                    else: # y == 5 and checkers > 1:
                        pixels[f"{x}{y}"] = f"({checkers}".rjust(3) + ")"
                    checkers -= 1
                else:
                    pixels[f"{x}{y}"] = "    "
        pixels["dcs"] = f" {self.dice[0] or ' '}:{self.dice[1] or ' '} "

        for player in [0, 1]:
            pixel = self._render_player(player=player)
            for idx in range(15):
                h = pixel + "h" + chr(ord("a") + idx)
                pixels[h] = f"  {pixel}  " if self.home[player] > idx else "     "

        pixels["ost"] = "     "
        pixels["xst"] = "     "
        if self.step == Game.Step.ROLL:
            pixels["ost" if self.pturn == 0 else "xst"] = " ROL "
        elif self.step == Game.Step.TURN:
            pixels["ost" if self.pturn == 0 else "xst"] = f" ({len([d for d in self.dice if d])}) "
        elif self.step == Game.Step.FINISHED:
            pixels["ost" if self.pturn == 0 else "xst"] = " WIN "

        return template.format(**pixels)

In [7]:
def validate(game: Game):
    p1, p2 = 0, 0
    for i in range(len(game.board)):
        if game.board[i] > 0:
            p1 += game.board[i]
        if game.board[i] < 0:
            p2 -= game.board[i]
    p1 += game.home[0]
    p2 += game.home[1]
    assert (p1, p2) == (15, 15), "invalid # of checkers at the board"

In [8]:
def random_move(game: Game):
    if not game.is_finished():
        moves = list(game._enum_valid_moves())
        if len(moves) > 0:
            pos, steps = random.choice(moves)
            game.turn(pos, steps)
            return True
        else:
            game.skip()
    return False

def auto_turn(game: Game):
    game.roll()
    for _ in range(len(game.dice)):
        if not random_move(game):
            break
    return game

def auto_rollout(game, turns: int = 100):
    for turn in range(turns):
        auto_turn(game)
        validate(game)
        if game.is_finished():
            break
    return game

In [9]:
g = Game(seed=42)
auto_rollout(g.start(), 100)


        |  o  | 24 | 23 | 22 | 21 | 20 | 19 |     | 18 | 17 | 16 | 15 | 14 | 13 |     |
        |  o  |-----------------------------|     |-----------------------------|     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |-----------------------------|  :  |-----------------------------|     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    | (4)|     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |  x |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |  x |  x |     |
        |  o  |    |    |    | 

## Test cases:

#### Can't make any moves, only skip is possible:

#### Check limitations on prime (blocking 6 in a row is not allowed if no opposing checker is in front)

In [10]:
import unittest

In [11]:
class Test(unittest.TestCase):
    def test_skips(self):
        g = Game(seed=42)
        g.start().roll().turn(1, 6).turn(7, 1)
        g.roll().turn(13, 3).turn(16, 2)
        g.roll().turn(1, 2).turn(3, 2).turn(5, 2).turn(8, 2)
        g.roll().turn(13,6).turn(18,1)
        g.roll().turn(10, 6).turn(16,6).turn(1, 6) # < there's no moves for red here
        with self.assertRaises(RuntimeError):
            g.turn(1, 6)
        g.skip()

    def test_blocking_primes(self):
        g = Game(seed=42)
        auto_turn(g.start().roll(1,4).turn(1, 4).turn(5, 1))
        auto_turn(g.roll(1,1).turn(1,1).turn(1,1).turn(2, 1).turn(3,1))
        auto_turn(g.roll(1,3).turn(1,1).turn(2,3))
        g.roll(1,2)
        with self.assertRaises(RuntimeError):
            g.turn(2,1)

    def test_nonblocking_primes(self):
        g = Game(seed=42)
        auto_turn(g.start().roll(1,4).turn(1, 4).turn(5, 1))
        auto_turn(g.roll(1,1).turn(1,1).turn(1,1).turn(2, 1).turn(3,1))
        g.roll(1,3).turn(1,1).turn(2,3)
        g.roll(3,6).turn(24,3).turn(3,6)
        g.roll(1,2).turn(1,2)

In [12]:
# Test().test_skips()
# Test().test_blocking_primes()
# Test().test_nonblocking_primes()

In [13]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_blocking_primes (__main__.Test) ... ok
test_nonblocking_primes (__main__.Test) ... ok
test_skips (__main__.Test) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


## Playing around

In [14]:
g = Game(seed=1761854789)
auto_rollout(g.start(), turns=100)


        |  o  | 24 | 23 | 22 | 21 | 20 | 19 |     | 18 | 17 | 16 | 15 | 14 | 13 |     |
        |  o  |-----------------------------|     |-----------------------------|     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |  x  |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |  x  |
        |  o  |-----------------------------|  :6 |-----------------------------|  x  |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |  x  |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |  x  |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |  x  |
        |  o  |    |    |    | 

In [15]:
g = Game()
auto_rollout(g.start(), turns=3)


        |     | 24 | 23 | 22 | 21 | 20 | 19 |     | 18 | 17 | 16 | 15 | 14 | 13 |     |
        |     |-----------------------------|     |-----------------------------|     |
        |     |    |  x |    |    |    |    |     |    |    |    |    |    |  x |     |
        |     |    |    |    |    |    |    |     |    |    |    |    |    |  x |     |
        |     |    |    |    |    |    |    |     |    |    |    |    |    |  x |     |
        |     |    |    |    |    |    |    |     |    |    |    |    |    |  x |     |
        |     |    |    |    |    |    |    |     |    |    |    |    |    |(10)|     |
        |     |-----------------------------|  :  |-----------------------------|     |
        |     |(10)|    |    |    |    |    |     |    |    |    |    |    |    |     |
        |     |  o |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |     |  o |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |     |  o |    |    | 

In [16]:
auto_rollout(Game().start(), turns=500)


        |  o  | 24 | 23 | 22 | 21 | 20 | 19 |     | 18 | 17 | 16 | 15 | 14 | 13 |     |
        |  o  |-----------------------------|     |-----------------------------|     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |    |    |     |
        |  o  |-----------------------------|  :6 |-----------------------------|     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    | (3)| (2)|     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |  x |  x |     |
        |  o  |    |    |    |    |    |    |     |    |    |    |    |  x |  x |     |
        |  o  |    |    |    | 

In [17]:
np.array([auto_rollout(Game().start(), turns=500).pturn for _ in tqdm.trange(1000)]).mean()

100%|██████████| 1000/1000 [00:14<00:00, 66.75it/s]


np.float64(0.513)

In [18]:
class Policy:
    def make_move(self, game: Game) -> bool:
        pass

    def play_turn(self, game: Game) -> Game:
        game.roll()
        logger.debug(f"t={game.t}, p={game.pturn} rolls {game.dice}")
        for _ in range(len(game.dice)):
            if not self._make_move(game):
                break
        return game

class RandomPolicy(Policy):
    def _make_move(self, game: Game):
        if not game.is_finished():
            moves = list(game._enum_valid_moves())
            if len(moves) > 0:
                pos, steps = random.choice(moves)
                game.turn(pos, steps)
                return True
            else:
                game.skip()
        return False

class LazyPolicy(Policy):
    def _first_move(self, game: Game) -> Optional[Tuple[int, int]]:
        try:
            return next(iter(game._enum_valid_moves()))
        except StopIteration:
            return None

    def _make_move(self, game: Game):
        if not game.is_finished():
            move = self._first_move(game)
            if move:
                pos, steps = move
                game.turn(pos, steps)
                return True
            else:
                game.skip()
        return False

@dataclass
class Results:
    winner: int
    turns: int
    reward: int

def calc_reward(game: Game) -> int:
    opponent = 1 if game.pturn == 0 else 0
    return 2 if game.home[opponent] == 0 else 1


def tournament(game: Game, player1: Policy, player2: Policy, turns: int = 100) -> Results:
    game.start()   
    for turn in range(turns):
        if game.is_finished():
            break
        cur_player = player1 if game.pturn == 0 else player2
        cur_player.play_turn(game)
    return Results(winner=game.pturn, turns=game.t, reward=calc_reward(game))

In [19]:
p1 = LazyPolicy()
p2 = LazyPolicy()
tournament(Game(), LazyPolicy(), LazyPolicy(), turns=1000)

Results(winner=0, turns=87, reward=1)

In [20]:
pd.DataFrame([tournament(Game(), LazyPolicy(), LazyPolicy(), turns=1000).__dict__ for _ in tqdm.trange(10000)]).mean()

100%|██████████| 10000/10000 [00:43<00:00, 232.28it/s]


winner     0.5522
turns     92.0029
reward     1.0761
dtype: float64

In [21]:
pd.DataFrame([tournament(Game(), RandomPolicy(), RandomPolicy(), turns=1000).__dict__ for _ in tqdm.trange(10000)]).mean()

100%|██████████| 10000/10000 [02:23<00:00, 69.64it/s]


winner     0.5029
turns     95.4575
reward     1.1456
dtype: float64

In [22]:
pd.DataFrame([tournament(Game(), RandomPolicy(), LazyPolicy(), turns=1000).__dict__ for _ in tqdm.trange(10000)]).mean()

100%|██████████| 10000/10000 [01:36<00:00, 104.08it/s]


winner     0.5019
turns     93.1297
reward     1.1396
dtype: float64